In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [36]:
df = pd.read_csv('../data/dataframe.csv')

**Columns have been shifted**

In [37]:
df.columns

Index(['Date', 'Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x',
       'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x',
       'CodeSum_x', 'Depth_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x',
       'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y',
       'Tavg_y', 'Depart_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y',
       'Sunrise_y', 'Sunset_y', 'CodeSum_y', 'Depth_y', 'PrecipTotal_y',
       'StnPressure_y', 'SeaLevel_y', 'ResultSpeed_y', 'ResultDir_y',
       'AvgSpeed_y', 'Day_length'],
      dtype='object')

In [41]:
plt.figure(figsize=(15,15))
df.corr()

,Unnamed: 0,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Tmax_x,Tmin_x,Tavg_x,...,Sunrise_y,Sunset_y,Depth_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,Day_length
Unnamed: 0,1.000000,-0.008781,0.073176,-0.120181,0.052858,0.096683,0.069694,-0.132672,-0.161358,-0.106543,...,NaN,NaN,NaN,-0.145318,0.159937,0.181264,0.046026,-0.141417,0.006116,-0.023218
Block,-0.008781,1.000000,0.045526,-0.070550,0.157320,-0.051671,0.009204,0.029824,0.029780,0.021061,...,NaN,NaN,NaN,-0.001165,-0.016086,-0.016144,0.000329,0.017317,0.000542,0.024390
Latitude,0.073176,0.045526,1.000000,-0.673251,0.368345,-0.056813,0.033330,-0.034916,-0.050166,-0.073627,...,NaN,NaN,NaN,-0.022353,-0.001144,0.005325,0.004976,-0.052353,0.003162,-0.010612
Longitude,-0.120181,-0.070550,-0.673251,1.000000,-0.404015,-0.070893,-0.068600,0.061790,0.067626,0.079558,...,NaN,NaN,NaN,0.029131,-0.006320,-0.014493,-0.001899,0.065015,0.001228,0.020279
AddressAccuracy,0.052858,0.157320,0.368345,-0.404015,1.000000,-0.067869,0.012268,-0.031277,-0.042987,-0.036712,...,NaN,NaN,NaN,-0.011174,-0.000749,0.003428,0.003665,-0.035287,0.001290,-0.012653
NumMosquitos,0.096683,-0.051671,-0.056813,-0.070893,-0.067869,1.000000,0.248242,0.110993,0.123207,0.075183,...,NaN,NaN,NaN,-0.003700,-0.014049,-0.023398,-0.019868,0.080735,-0.014457,0.124736
WnvPresent,0.069694,0.009204,0.033330,-0.068600,0.012268,0.248242,1.000000,0.055562,0.078574,0.084429,...,NaN,NaN,NaN,0.029507,0.010017,0.005207,-0.083813,0.005391,-0.087089,0.072590
Tmax_x,-0.132672,0.029824,-0.034916,0.061790,-0.031277,0.110993,0.055562,1.000000,0.748719,0.294843,...,NaN,NaN,NaN,-0.062828,-0.275523,-0.316708,-0.047185,0.179642,0.012468,0.480950
Tmin_x,-0.161358,0.029780,-0.050166,0.067626,-0.042987,0.123207,0.078574,0.748719,1.000000,0.322566,...,NaN,NaN,NaN,0.071644,-0.409629,-0.458553,-0.123329,0.222172,-0.120058,0.567003
Tavg_x,-0.106543,0.021061,-0.073627,0.079558,-0.036712,0.075183,0.084429,0.294843,0.322566,1.000000,...,NaN,NaN,NaN,0.046633,0.138794,0.116091,-0.184282,0.045040,-0.169227,0.461609


<Figure size 1080x1080 with 0 Axes>

In [19]:
df['Day_length'].index[84:]

RangeIndex(start=84, stop=9693, step=1)


In [42]:
x = np.corrcoef(df['Tavg_x'].rolling(3).mean().shift(14).dropna(), df['NumMosquitos'].drop(df.index[0:16]))

In [47]:
x

array([[1.        , 0.06361912],
       [0.06361912, 1.        ]])

In [46]:
x[0,1]

0.06361911839904065

In [ ]:
def hi_corr(f1, f2, rolling_mean=3, min_val=1, max_val=30):
    corr_array = []
    
    